In [1]:
import pandas as pd 
import networkx as nx
import itertools
from matplotlib.patches import Polygon
import operator
from matplotlib.collections import PatchCollection
import decimal
import pickle as pkl
import plotly.graph_objects as go
import os
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)
import numpy as np
import sys
import matplotlib.pyplot as plt


new_path=os.getcwd()
new_path =new_path[:-13]
if new_path not in sys.path:
    sys.path.append(new_path)
#----
import continuous2graph
import pre_extraction
import quality_measure
import terminal_computation 
import filtering
import utils
#----

# PlotlyVisualizationModule 11: *Partition of $[0,1]^2$*.

## Loading Graphs

In [2]:
path_= './PVM_data/1_b14_6dv_sf14_rect_cnstrect_cnst/tdens_t1E-03_graph1_wmER.dat'
with open(path_, 'rb') as file:
    G_pre_extracted=pkl.load(file)

In [3]:
path_= './PVM_data/1_b14_6dv_sf14_rect_cnstrect_cnst/filtered_tdens_t1E-03_graph1.dat'
with open(path_, 'rb') as file:
    G_filtered=pkl.load(file)

## Loading optimal weights

In [4]:
folder_name='./PVM_data/1_b14_6dv_sf14_rect_cnstrect_cnst/'
min_=.001
funct = 'tdens'

G_bar, G_triang, dict_seq = continuous2graph.preprocessing_cont(
	folder_name, min_, funct
)

get_baryc
--- 0.016605854034423828 seconds ---
bar2dict
--- 0.0006330013275146484 seconds ---
extracting_weights
--- 0.011535167694091797 seconds ---
weight2dict
FILE_NAME output/result/opt_tdens.dat
The max tdens in this case is equal to 1.851672168 . The chosen threshold percentage is 0.1 %
Numerical threshold: 0.001851672168
--- 0.0009500980377197266 seconds ---
completing_with_zeros
--- 9.059906005859375e-06 seconds ---
dict2graph
--- 0.0002961158752441406 seconds ---
getting_grid
--- 0.012432336807250977 seconds ---
saving_the_grid_to_dict
--- 0.000240325927734375 seconds ---
defining_graph_with_the_nodes_of_the_grid
--- 0.00011658668518066406 seconds ---
getting_the_edges_of_the_grid
--- 0.0011630058288574219 seconds ---


## Defining the traces

In [5]:
max_=max([G_bar.nodes[node]['weight'] for node in G_bar.nodes()])

In [6]:
weights=[entry[1] for entry in G_bar.nodes(data='weight') if entry[1]>min_*max_]

In [7]:
edge_trace_triang = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G_triang.edges():
    x0, y0 = G_triang.node[edge[0]]['pos']
    x1, y1 = G_triang.node[edge[1]]['pos']
    edge_trace_triang['x'] += tuple([x0, x1, None])
    edge_trace_triang['y'] += tuple([y0, y1, None])

node_trace_bar_weights= go.Scatter(
    x=[],
    y=[],
    text=weights,
    mode='markers',
    #hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=False,
        color=weights,#<---------------------- new!
        size=8,
        colorbar=dict(
            thickness=5,
            title='Weights (tdens)',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G_bar.nodes():
    if G_bar.node[node]['weight']>min_*max_:
        x, y = G_bar.node[node]['pos']
        node_trace_bar_weights['x'] += tuple([x])
        node_trace_bar_weights['y'] += tuple([y])

x1 , y1 , x2 , y2 = 1.0/8.0,1/4, 3/8, 3/4
x3 , y3 , x4 , y4 = 5.0/8.0,1/4, 7/8, 3/4    

source =go.Scatter(x=[x1,x1,x2,x2], y=[y1,y2,y2,y1], fill="toself",fillcolor="green",opacity=0.05)    
sink =go.Scatter(x=[x3,x3,x4,x4], y=[y3,y4,y4,y3], fill="toself",fillcolor="red",opacity=0.05)
    
edge_trace_pre= go.Scatter(
    x=[],
    y=[],
    line=dict(width=1.5,color='black'),
    hoverinfo='none',
    mode='lines')

for edge in G_pre_extracted.edges():
    x0, y0 = G_pre_extracted.node[edge[0]]['pos']
    x1, y1 = G_pre_extracted.node[edge[1]]['pos']
    edge_trace_pre['x'] += tuple([x0, x1, None])
    edge_trace_pre['y'] += tuple([y0, y1, None])
    
edge_filtered= go.Scatter(
    x=[],
    y=[],
    line=dict(width=2,color='blue'),
    hoverinfo='none',
    mode='lines')

for edge in G_filtered.edges():
    x0, y0 = G_filtered.node[edge[0]]['pos']
    x1, y1 = G_filtered.node[edge[1]]['pos']
    edge_filtered['x'] += tuple([x0, x1, None])
    edge_filtered['y'] += tuple([y0, y1, None])
    

In [8]:
fig = make_subplots(rows=1, cols=2)

basic_plots = [source,sink,
                       edge_trace_triang,
             node_trace_bar_weights]

for plot in  basic_plots+ [edge_trace_pre]:
    
    fig.add_trace(plot,
    row=1, col=1
)


for plot in  basic_plots + [edge_filtered]:
    
    fig.add_trace(plot,
    row=1, col=2
)

fig.update_layout(height=550, width=900, title_text="Side By Side Subplots",
                 paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [9]:
edge_trace_grid ={}

for N in [5,8,11]:
    _,new_grid=quality_measure.partition(N)


    edge_trace_grid[N] = go.Scatter(
        x=[],
        y=[],
        text=list(new_grid.edges(data=True)),
        line=dict(width=3,color='red'),
        hoverinfo='text',
        mode='lines',
    opacity=0.5)

    for edge in new_grid.edges():
        x0, y0 = new_grid.node[edge[0]]['pos']
        x1, y1 = new_grid.node[edge[1]]['pos']
        edge_trace_grid[N]['x'] += tuple([x0, x1, None])
        edge_trace_grid[N]['y'] += tuple([y0, y1, None])

## Getting the partitions

In [10]:

for N in [5,8,11]:
    fig = make_subplots(rows=1, cols=2)
    for plot in  [edge_trace_grid[N],edge_trace_pre]+basic_plots :
        
        fig.add_trace(plot,
        row=1, col=1
    )


    for plot in  [edge_trace_grid[N],edge_filtered]+basic_plots:

        fig.add_trace(plot,
        row=1, col=2
    )

    fig.update_layout(height=550, width=900,
                     paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)')
    fig.show()

# PlotlyVisualizationModule 12: *Weight and length*.

## Computing the weight-related part and the length-related part of the quality measure

In [11]:
qm_pre_extr = {}
qm_filtered = {}

print('\n      N    | Pre-extracted Graph    |      Filtered Graph      |')
print('           |       qw   |  L        |        qw   |  L         |')
for N in [5,8,11]:
    
    # pre-extracted graph
    qm_pre_extr[N] = {}
    
    part_dict,_,_ = quality_measure.partition_set(N)
    
    
    q_measure, q_weight, L, _,_ = quality_measure.q_measure(G_pre_extracted,
                                                           G_bar, 
                                                           part_dict, 
                                                           'l2', min_=min_)
    qm_pre_extr[N]['qw'] = q_weight
    qm_pre_extr[N]['L'] = L
    
    # filtered
    qm_filtered[N] = {}
    
    part_dict,_,_ = quality_measure.partition_set(N)
    
    
    q_measure, q_weight, L, _,_ = quality_measure.q_measure(G_filtered,
                                                           G_bar, 
                                                           part_dict, 
                                                           'l2',
                                                           min_=min_)
    qm_filtered[N]['qw'] = q_weight
    qm_filtered[N]['L'] = L
    
    print("%10.1e"%N,'|',"%10.3e"% qm_pre_extr[N]['qw'],
          "%10.3e "% qm_pre_extr[N]['L'],'|',
          "%10.3e "% qm_filtered[N]['qw'],
          "%10.3e "% qm_filtered[N]['L'],' |')


      N    | Pre-extracted Graph    |      Filtered Graph      |
           |       qw   |  L        |        qw   |  L         |
   5.0e+00 |  6.347e-02  1.594e+01  |  1.791e-01   3.100e+00   |
   8.0e+00 |  2.520e-02  1.594e+01  |  6.114e-02   3.100e+00   |
   1.1e+01 |  1.269e-02  1.594e+01  |  2.521e-02   3.100e+00   |


# PlotlyVisualizationModule 13: *Quality measure*.

## Comparing to other toy graphs

In [12]:
#first toy graph

G_pre_extracted_shifted = G_pre_extracted.copy()

delta=(.1,.2)

for n in G_pre_extracted_shifted.nodes():
    G_pre_extracted_shifted.node[n]['pos'] = G_pre_extracted_shifted.node[n]['pos']+delta

In [13]:
# second toy graph

G_filtered_shifted = G_filtered.copy()

a=.2

for n in G_filtered_shifted.nodes():
    G_filtered_shifted.node[n]['pos'] = a*G_filtered_shifted.node[n]['pos']

In [14]:
# measures

graph_list = [G_pre_extracted,G_filtered,G_pre_extracted_shifted,G_filtered_shifted]
qm=[]
qw=[]
L_list=[]
for graph in graph_list:
    
    q_measure, q_weight, L, triang_weight_vector,weight_difference_vector = quality_measure.q_measure(graph,
                                                               G_bar, 
                                                               part_dict, 
                                                               'l1',
                                                               min_=min_)
    
    qm.append(q_measure)
    qw.append(q_weight)
    L_list.append(L)

In [15]:
#re scaling

max_qm = max(qm)
max_qw = max(qw)
max_L = max(L_list)


qw_normal=[]
L_normal=[]

for i in range(4):
    qw_normal.append(qw[i]/max_qw)
    L_normal.append(L_list[i]/max_L)

qm=[]
qw=[]
L_list=[]

for i in range(4):
    val_qm = "%10.3e"% (qw_normal[i]+L_normal[i])
    val_qw = "%10.3e"% qw_normal[i]
    val_L = "%10.3e"% L_normal[i]
    
    qm.append(val_qm)
    qw.append(val_qw)
    L_list.append(val_L)

In [16]:

edge_pre_shifted= go.Scatter(
    x=[],
    y=[],
    line=dict(width=1.5,color='black'),
    hoverinfo='none',
    mode='lines')

for edge in G_pre_extracted_shifted.edges():
    x0, y0 = G_pre_extracted_shifted.node[edge[0]]['pos']
    x1, y1 = G_pre_extracted_shifted.node[edge[1]]['pos']
    edge_pre_shifted['x'] += tuple([x0, x1, None])
    edge_pre_shifted['y'] += tuple([y0, y1, None])
    
edge_filtered_shifted= go.Scatter(
    x=[],
    y=[],
    line=dict(width=2,color='blue'),
    hoverinfo='none',
    mode='lines')

for edge in G_filtered_shifted.edges():
    x0, y0 = G_filtered_shifted.node[edge[0]]['pos']
    x1, y1 = G_filtered_shifted.node[edge[1]]['pos']
    edge_filtered_shifted['x'] += tuple([x0, x1, None])
    edge_filtered_shifted['y'] += tuple([y0, y1, None])




fig = make_subplots(rows=2, cols=2
                   ,subplot_titles=(qm[0]+"(qm) =" +qw[0]+"(qw)+"+L_list[0]+"(L)",
                                    qm[1]+"(qm) =" +qw[1]+"(qw)+"+L_list[1]+"(L)",
                                    qm[2]+"(qm) =" +qw[2]+"(qw)+"+L_list[2]+"(L)",
                                    qm[3]+"(qm) =" +qw[3]+"(qw)+"+L_list[3]+"(L)"))

basic_plots = [source,sink,
                       edge_trace_triang,
             node_trace_bar_weights]

for plot in  [edge_trace_grid[N]]+basic_plots+ [edge_trace_pre]:
    
    fig.add_trace(plot,
    row=1, col=1
)


for plot in  [edge_trace_grid[N]]+basic_plots + [edge_filtered]:
    
    fig.add_trace(plot,
    row=1, col=2
)
    
for plot in  [edge_trace_grid[N]]+basic_plots+ [edge_pre_shifted]:
    
    fig.add_trace(plot,
    row=2, col=1
)


for plot in  [edge_trace_grid[N]]+basic_plots + [edge_filtered_shifted]:
    
    fig.add_trace(plot,
    row=2, col=2
)

fig.update_layout(height=2*550, width=900, title_text="Side By Side Subplots",
                 paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)')
fig.show()